In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
import mglearn
from sklearn.preprocessing import OneHotEncoder


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD, Adam

ModuleNotFoundError: No module named 'mglearn'

In [ ]:
# FONT 깨질때 폰트깨질때
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname = "C:/Windows/Fonts/malgun.ttf").get_name()
rc('font',family=font_name)

In [ ]:
df = pd.read_excel('./최종 데이터 그래프그리기용.xlsx') 
print(df)

psqi_df=df[['Sample_ID','GROUP','COHORT','SEX','AGE','TG_1','FatPercentage _1','BMI_1','PSQI_TOTAL_1','TG_2','FatPercentage_2','BMI_2','PSQI_TOTAL_2']]

psqi_df=df[['Sample_ID','GROUP','COHORT','SEX','AGE','Insulin _1','FatPercentage _1','TG_1','BMI_1','AST_1','BUN_1',
            'PSQI_TOTAL_1','PSQI_C1_1','PSQI_C2_1','PSQI_C3_1','PSQI_C4_1','PSQI_C5_1','PSQI_C6_1','PSQI_C7_1','PSQI_Q1_1','PSQI_Q2_1','PSQI_Q3_1','PSQI_Q4_1',
            'PSQI_Q5a_1','PSQI_Q5b_1','PSQI_Q5c_1','PSQI_Q5d_1','PSQI_Q5e_1','PSQI_Q5f_1','PSQI_Q5g_1','PSQI_Q5h_1','PSQI_Q5i_1','PSQI_Q5j_1','PSQI_Q6_1','PSQI_Q7_1','PSQI_Q8_1','PSQI_Q9_1',
              'Insulin _2','FatPercentage_2','TG_2','BMI_2','AST_2','BUN_2','PSQI_TOTAL_2','PSQI_C1_2','PSQI_C2_2','PSQI_C3_2','PSQI_C4_2','PSQI_C5_2','PSQI_C6_2','PSQI_C7_2','PSQI_Q1_2','PSQI_Q2_2','PSQI_Q3_2','PSQI_Q4_2',
            'PSQI_Q5a_2','PSQI_Q5b_2','PSQI_Q5c_2','PSQI_Q5d_2','PSQI_Q5e_2','PSQI_Q5f_2','PSQI_Q5g_2','PSQI_Q5h_2','PSQI_Q5i_2','PSQI_Q5j_2','PSQI_Q6_2','PSQI_Q7_2','PSQI_Q8_2','PSQI_Q9_2']]

psqi_df=df[['Sample_ID','GROUP','COHORT','SEX','AGE','HDL_1','DBP_1','Waist_1','SBP_1','BMI_1','Fat_1_x','PSQI_TOTAL_1','TG_1',
           'HDL_2','DBP_2','Waist_2','SBP_2','BMI_2','Fat_2_x','PSQI_TOTAL_2','TG_2']]

In [ ]:
psqi_df=df[['Sample_ID','GROUP','COHORT','SEX','AGE','Insulin _1','FatPercentage _1','TG_1','BMI_1','AST_1','BUN_1','HDL_1','DBP_1','Waist_1','SBP_1','Fat_1_x','LDL_1',
            'PSQI_TOTAL_1','PSQI_C1_1','PSQI_C2_1','PSQI_C3_1','PSQI_C4_1','PSQI_C5_1','PSQI_C6_1','PSQI_C7_1','PSQI_Q1_1','PSQI_Q2_1','PSQI_Q3_1','PSQI_Q4_1',
            'PSQI_Q5a_1','PSQI_Q5b_1','PSQI_Q5c_1','PSQI_Q5d_1','PSQI_Q5e_1','PSQI_Q5f_1','PSQI_Q5g_1','PSQI_Q5h_1','PSQI_Q5i_1','PSQI_Q5j_1','PSQI_Q6_1','PSQI_Q7_1','PSQI_Q8_1','PSQI_Q9_1',
              'Insulin _2','FatPercentage_2','TG_2','BMI_2','AST_2','BUN_2','HDL_2','DBP_2','Waist_2','SBP_2','Fat_2_x','LDL_2',
            'PSQI_TOTAL_2','PSQI_C1_2','PSQI_C2_2','PSQI_C3_2','PSQI_C4_2','PSQI_C5_2','PSQI_C6_2','PSQI_C7_2','PSQI_Q1_2','PSQI_Q2_2','PSQI_Q3_2','PSQI_Q4_2',
            'PSQI_Q5a_2','PSQI_Q5b_2','PSQI_Q5c_2','PSQI_Q5d_2','PSQI_Q5e_2','PSQI_Q5f_2','PSQI_Q5g_2','PSQI_Q5h_2','PSQI_Q5i_2','PSQI_Q5j_2','PSQI_Q6_2','PSQI_Q7_2','PSQI_Q8_2','PSQI_Q9_2']]

In [ ]:
psqi_df

In [ ]:
#보조 호르몬 요법을 받고 있는 Cohort 3 제거 Filter 적용
psqi_df = psqi_df[(psqi_df['COHORT'] != 3)]
psqi_df

In [ ]:
psqi_df=psqi_df.dropna()
psqi_df.reset_index(drop=True, inplace=True)
psqi_df.isnull().sum()
psqi_df=psqi_df.drop(["Sample_ID", "GROUP", "COHORT"],axis=1)
#1분, 매일다름, 정해진간이없음 삭제

In [ ]:
#<150 (정상) : 0 -> 중간 값인 80으로 조정
#한단계씩 상향 조정
#150~199 (약간 높음 - 체중감량과 운동요법) : 1 -> 80~150 (정상)
#200~499 (높음 - 생활요법과 함께 약물치료 고려) : 2 -> 150~199 (약간 높음)
#>500 (아주 높음 - 즉시 약물치료) : 3 한명도 해당 X -> 200~ (높음~아주높음)

#psqi_df["TG_1"] = psqi_df["TG_1"].apply(lambda x: 0. if x<80 else 1. if 80<=x<150 else 2. if 150<=x<200 else 3.)
#psqi_df["TG_2"] = psqi_df["TG_2"].apply(lambda x: 0. if x<80 else 1. if 80<=x<150 else 2. if 150<=x<200 else 3.)

In [ ]:
psqi_df["SEX"] = psqi_df["SEX"].apply(lambda x: 1. if x=='M' else 0.)

In [ ]:
psqi_df["Insulin _1"] = psqi_df["Insulin _1"].apply(lambda x: 0.1 if x=='<0.2' else 0. if x=='<0.1' else x)
psqi_df["Insulin _2"] = psqi_df["Insulin _1"].apply(lambda x: 0.1 if x=='<0.2' else 0. if x=='<0.1' else x)

#x 배열 생성 (x=psqi)
X1=psqi_df[['PSQI_TOTAL_1','PSQI_C1_1','PSQI_C2_1','PSQI_C3_1','PSQI_C4_1','PSQI_C5_1','PSQI_C6_1','PSQI_C7_1',
                           'PSQI_Q1_1','PSQI_Q2_1','PSQI_Q3_1','PSQI_Q4_1','PSQI_Q5a_1','PSQI_Q5b_1','PSQI_Q5c_1','PSQI_Q5d_1','PSQI_Q5e_1','PSQI_Q5f_1','PSQI_Q5g_1','PSQI_Q5h_1','PSQI_Q5i_1','PSQI_Q5j_1',
                           'PSQI_Q6_1','PSQI_Q7_1','PSQI_Q8_1','PSQI_Q9_1']].values
X2=psqi_df[['PSQI_TOTAL_2','PSQI_C1_2','PSQI_C2_2','PSQI_C3_2','PSQI_C4_2','PSQI_C5_2','PSQI_C6_2','PSQI_C7_2',
                           'PSQI_Q1_2','PSQI_Q2_2','PSQI_Q3_2','PSQI_Q4_2','PSQI_Q5a_2','PSQI_Q5b_2','PSQI_Q5c_2','PSQI_Q5d_2','PSQI_Q5e_2','PSQI_Q5f_2','PSQI_Q5g_2','PSQI_Q5h_2','PSQI_Q5i_2','PSQI_Q5j_2',
                           'PSQI_Q6_2','PSQI_Q7_2','PSQI_Q8_2','PSQI_Q9_2']].values

#y 배열 생성 (y=tg)
Y1 = psqi_df['TG_1'].tolist()
Y2 = psqi_df['TG_2'].tolist()
Y=Y1+Y2

#x 배열 생성 (x=임의)
X1=psqi_df[['SEX','AGE','Insulin _1','FatPercentage _1','BMI_1','AST_1','BUN_1','PSQI_TOTAL_1','PSQI_C1_1','PSQI_C2_1','PSQI_C3_1','PSQI_C4_1','PSQI_C5_1','PSQI_C6_1','PSQI_C7_1',
            'PSQI_Q1_1','PSQI_Q2_1','PSQI_Q3_1','PSQI_Q4_1','PSQI_Q5a_1','PSQI_Q5b_1','PSQI_Q5c_1','PSQI_Q5d_1','PSQI_Q5e_1','PSQI_Q5f_1','PSQI_Q5g_1','PSQI_Q5h_1','PSQI_Q5i_1','PSQI_Q5j_1',
            'PSQI_Q6_1','PSQI_Q7_1','PSQI_Q8_1','PSQI_Q9_1']].values
X2=psqi_df[['SEX','AGE','Insulin _2','FatPercentage_2','BMI_2','AST_2','BUN_2','PSQI_TOTAL_2','PSQI_C1_2','PSQI_C2_2','PSQI_C3_2','PSQI_C4_2','PSQI_C5_2','PSQI_C6_2','PSQI_C7_2',
            'PSQI_Q1_2','PSQI_Q2_2','PSQI_Q3_2','PSQI_Q4_2','PSQI_Q5a_2','PSQI_Q5b_2','PSQI_Q5c_2','PSQI_Q5d_2','PSQI_Q5e_2','PSQI_Q5f_2','PSQI_Q5g_2','PSQI_Q5h_2','PSQI_Q5i_2','PSQI_Q5j_2',
            'PSQI_Q6_2','PSQI_Q7_2','PSQI_Q8_2','PSQI_Q9_2']].values

#y 배열 생성 (y=tg)
Y1 = psqi_df['TG_1'].tolist()
Y2 = psqi_df['TG_2'].tolist()
Y=Y1+Y2

#x 배열 생성 (x=선별)
X1=psqi_df[['AGE','HDL_1','DBP_1','Waist_1','SBP_1','BMI_1','Fat_1_x','PSQI_TOTAL_1']].values
X2=psqi_df[['AGE','HDL_2','DBP_2','Waist_2','SBP_2','BMI_2','Fat_2_x','PSQI_TOTAL_2']].values
X=np.concatenate((X1, X2), axis=0)

#y 배열 생성 (y=tg)
Y1 = psqi_df['TG_1'].tolist()
Y2 = psqi_df['TG_2'].tolist()
Y=Y1+Y2

#x 배열 생성 (x=임의+선별)
X1=psqi_df[['HDL_1','DBP_1','Waist_1','SBP_1','Fat_1_x','Insulin _1','FatPercentage _1','BMI_1','AST_1','BUN_1','LDL_1',
            'PSQI_TOTAL_1','PSQI_C1_1','PSQI_C2_1','PSQI_C3_1','PSQI_C4_1','PSQI_C5_1','PSQI_C6_1','PSQI_C7_1','PSQI_Q1_1','PSQI_Q2_1','PSQI_Q3_1','PSQI_Q4_1',
            'PSQI_Q5a_1','PSQI_Q5b_1','PSQI_Q5c_1','PSQI_Q5d_1','PSQI_Q5e_1','PSQI_Q5f_1','PSQI_Q5g_1','PSQI_Q5h_1','PSQI_Q5i_1','PSQI_Q5j_1','PSQI_Q6_1','PSQI_Q7_1','PSQI_Q8_1','PSQI_Q9_1']].values

X2=psqi_df[['HDL_2','DBP_2','Waist_2','SBP_2','Fat_2_x','Insulin _2','FatPercentage_2','BMI_2','AST_2','BUN_2','LDL_2',
            'PSQI_TOTAL_2','PSQI_C1_2','PSQI_C2_2','PSQI_C3_2','PSQI_C4_2','PSQI_C5_2','PSQI_C6_2','PSQI_C7_2','PSQI_Q1_2','PSQI_Q2_2','PSQI_Q3_2','PSQI_Q4_2',
            'PSQI_Q5a_2','PSQI_Q5b_2','PSQI_Q5c_2','PSQI_Q5d_2','PSQI_Q5e_2','PSQI_Q5f_2','PSQI_Q5g_2','PSQI_Q5h_2','PSQI_Q5i_2','PSQI_Q5j_2','PSQI_Q6_2','PSQI_Q7_2','PSQI_Q8_2','PSQI_Q9_2']].values
X=np.concatenate((X1, X2), axis=0)

#y 배열 생성 (y=tg)
Y1 = psqi_df['TG_1'].tolist()
Y2 = psqi_df['TG_2'].tolist()
Y=Y1+Y2

In [ ]:
#x 배열 생성 (x=임의+선별)
X1=psqi_df[['HDL_1','DBP_1','Waist_1','SBP_1','Fat_1_x','Insulin _1','FatPercentage _1','BMI_1','AST_1','BUN_1','LDL_1',]].values

X2=psqi_df[['HDL_2','DBP_2','Waist_2','SBP_2','Fat_2_x','Insulin _2','FatPercentage_2','BMI_2','AST_2','BUN_2','LDL_2',]].values
X=np.concatenate((X1, X2), axis=0)

#y 배열 생성 (y=tg, sex, age)
Y1= psqi_df[['TG_1']].values
Y2= psqi_df[['TG_2']].values
Y=np.concatenate((Y1, Y2), axis=0)

In [ ]:
#x 배열 생성 (x=임의+선별)
X1=psqi_df[['HDL_1','DBP_1','Waist_1','SBP_1','Fat_1_x','Insulin _1','FatPercentage _1','BMI_1','AST_1','BUN_1','LDL_1',
            'PSQI_TOTAL_1','PSQI_C1_1','PSQI_C2_1','PSQI_C3_1','PSQI_C4_1','PSQI_C5_1','PSQI_C6_1','PSQI_C7_1','PSQI_Q1_1','PSQI_Q2_1','PSQI_Q3_1','PSQI_Q4_1',
            'PSQI_Q5a_1','PSQI_Q5b_1','PSQI_Q5c_1','PSQI_Q5d_1','PSQI_Q5e_1','PSQI_Q5f_1','PSQI_Q5g_1','PSQI_Q5h_1','PSQI_Q5i_1','PSQI_Q5j_1','PSQI_Q6_1','PSQI_Q7_1','PSQI_Q8_1','PSQI_Q9_1']].values

X2=psqi_df[['HDL_2','DBP_2','Waist_2','SBP_2','Fat_2_x','Insulin _2','FatPercentage_2','BMI_2','AST_2','BUN_2','LDL_2',
            'PSQI_TOTAL_2','PSQI_C1_2','PSQI_C2_2','PSQI_C3_2','PSQI_C4_2','PSQI_C5_2','PSQI_C6_2','PSQI_C7_2','PSQI_Q1_2','PSQI_Q2_2','PSQI_Q3_2','PSQI_Q4_2',
            'PSQI_Q5a_2','PSQI_Q5b_2','PSQI_Q5c_2','PSQI_Q5d_2','PSQI_Q5e_2','PSQI_Q5f_2','PSQI_Q5g_2','PSQI_Q5h_2','PSQI_Q5i_2','PSQI_Q5j_2','PSQI_Q6_2','PSQI_Q7_2','PSQI_Q8_2','PSQI_Q9_2']].values
X=np.concatenate((X1, X2), axis=0)

#y 배열 생성 (y=tg, sex, age)
Y1= psqi_df[['TG_1']].values
Y2= psqi_df[['TG_2']].values
Y=np.concatenate((Y1, Y2), axis=0)

#x df 생성 (x=임의+선별) ~수정작업중~
X1=psqi_df[['HDL_1','DBP_1','Waist_1','SBP_1','Fat_1_x','Insulin _1','FatPercentage _1','BMI_1','AST_1','BUN_1','LDL_1',
            'PSQI_TOTAL_1','PSQI_C1_1','PSQI_C2_1','PSQI_C3_1','PSQI_C4_1','PSQI_C5_1','PSQI_C6_1','PSQI_C7_1','PSQI_Q1_1','PSQI_Q2_1','PSQI_Q3_1','PSQI_Q4_1',
            'PSQI_Q5a_1','PSQI_Q5b_1','PSQI_Q5c_1','PSQI_Q5d_1','PSQI_Q5e_1','PSQI_Q5f_1','PSQI_Q5g_1','PSQI_Q5h_1','PSQI_Q5i_1','PSQI_Q5j_1','PSQI_Q6_1','PSQI_Q7_1','PSQI_Q8_1','PSQI_Q9_1']]
#예외처리
X1=X1.rename({ 'Fat_1_x' : 'Fat_x_1'}, axis=1)
X1=X1.rename({ 'FatPercentage _1' : 'FatPercentage_1'}, axis=1)
X1=X1.rename({ 'Insulin _1' : 'Insulin_1'}, axis=1)
for i in X1.columns:
    X1=X1.rename({ i : i[:-2]}, axis=1)

X2=psqi_df[['HDL_2','DBP_2','Waist_2','SBP_2','Fat_2_x','Insulin _2','FatPercentage_2','BMI_2','AST_2','BUN_2','LDL_2',
            'PSQI_TOTAL_2','PSQI_C1_2','PSQI_C2_2','PSQI_C3_2','PSQI_C4_2','PSQI_C5_2','PSQI_C6_2','PSQI_C7_2','PSQI_Q1_2','PSQI_Q2_2','PSQI_Q3_2','PSQI_Q4_2',
            'PSQI_Q5a_2','PSQI_Q5b_2','PSQI_Q5c_2','PSQI_Q5d_2','PSQI_Q5e_2','PSQI_Q5f_2','PSQI_Q5g_2','PSQI_Q5h_2','PSQI_Q5i_2','PSQI_Q5j_2','PSQI_Q6_2','PSQI_Q7_2','PSQI_Q8_2','PSQI_Q9_2']]
#예외처리
X2=X2.rename({ 'Fat_2_x' : 'Fat_x_2'}, axis=1)
X2=X2.rename({ 'Insulin _2' : 'Insulin_2'}, axis=1)
for i in X2.columns:
    X2=X2.rename({ i : i[:-2]}, axis=1)

X=pd.concat((X1, X2), axis=0)


#y df 생성 (y=tg, sex, age)
Y1= psqi_df[['TG_1', 'SEX', 'AGE']]
Y2= psqi_df[['TG_2', 'SEX', 'AGE']]

Y1=Y1.rename({'TG_1' : 'TG'}, axis=1)
Y2=Y2.rename({'TG_2' : 'TG'}, axis=1)
Y=pd.concat((Y1, Y2), axis=0)

In [ ]:
len(X), len(Y)

In [ ]:
X.shape, Y.shape

# 정규화

In [ ]:
#정규화 (변수간의 스케일 차이)
X = StandardScaler().fit_transform(X)

#주성분 분석 실시
pca = decomposition.PCA(n_components=2) #주성분 갯수 2개 생성
sklearn_pca_x = pca.fit_transform(x_std)
sklearn_result = pd.DataFrame(sklearn_pca_x, columns=['PC1','PC2'])
sklearn_result

X=sklearn_result.values

In [ ]:
np.random.seed(0)
X=np.asarray(X).astype(np.float)
Y=np.asarray(Y).astype(np.float)
x_train,x_test,y_train,y_test=train_test_split(X,Y,train_size=0.8)

In [ ]:
X.shape, Y.shape

x_train

x_test

y_train

y_test

# MLP

In [ ]:
#define model
model=Sequential()
model.add(Dense(32, activation='relu', input_dim=11)) #전체 37, psqi빼면 전체 11
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='relu'))

model.compile(loss='mse',optimizer='rmsprop',metrics=['mse'])

#fit model
history=model.fit(x_train, y_train, epochs=100, batch_size=4)
model.evaluate(x_test, y_test) 

In [ ]:
model.summary()

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))

In [2]:
y_p = model.predict(x_test)
                    

#for t,p in zip(y_test,y_p):
#    print(t,p)

NameError: name 'model' is not defined

In [ ]:
y_test_list=np.array(y_test).flatten().tolist()
y_p_list=np.array(y_p).flatten().tolist()
y_train_list=np.array(y_train).flatten().tolist()

In [3]:
#오차 범위 10 설정

scores = 0
for i in range(len(y_test)):
    if  y_train_list[i]-10 <= y_p_list[i] <= y_train_list[i]+10:
        scores+=1

accuracy=scores/len(y_test)
print("train set prediction accuracy: {:.2f} %".format(accuracy*100)) # 예측 정확도

#======================================================================================

scores = 0
for i in range(len(y_test)):
    if  y_test_list[i]-10 <= y_p_list[i] <= y_test_list[i]+10:
        scores+=1

accuracy=scores/len(y_test)
print("test set prediction accuracy: {:.2f} %".format(accuracy*100)) # 예측 정확도

NameError: name 'y_test' is not defined

In [4]:
#오차 범위 20 설정

scores = 0
for i in range(len(y_test)):
    if  y_train_list[i]-20 <= y_p_list[i] <= y_train_list[i]+20:
        scores+=1

accuracy=scores/len(y_test)
print("train set prediction accuracy: {:.2f} %".format(accuracy*100)) # 예측 정확도

#======================================================================================

scores = 0
for i in range(len(y_test)):
    if  y_test_list[i]-20 <= y_p_list[i] <= y_test_list[i]+20:
        scores+=1

accuracy=scores/len(y_test)
print("test set prediction accuracy: {:.2f} %".format(accuracy*100)) # 예측 정확도

NameError: name 'y_test' is not defined

# 오차범위 별 허용 예측 정확도
### 1. 임의+선별
- <오차범위 +-10> <br>
train: 21.13% <br>
test: 32.39% <br>

- <오차범위 +-20> <br>
train: 33.80% <br>
test: 54.93% <br>

### 2. 임의+선별 (PSQI제외)
- <오차범위 +-10> <br>
train: 16.90% <br>
test: 12.68% <br>

- <오차범위 +-20> <br>
train: 33.80% <br>
test: 45.07% <br>